In [1]:
import requests
import folium
from folium.plugins import MarkerCluster
import geoip2.database
import pandas as pd
from nslookup import Nslookup
from netaddr import IPNetwork  # For CIDR expansion
import socket

# Initialize GeoIP Reader and Nslookup
geoip_reader = geoip2.database.Reader('GeoLite2-City.mmdb')
dns_query = Nslookup(dns_servers=["8.8.8.8"])

# Fetch the CIDR list from the URL
url = "https://raw.githubusercontent.com/threatcode/proxy-list/refs/heads/master/COUNTRY_CIDR/BD_CIDR.txt"
response = requests.get(url)

# Ensure the request was successful
if response.status_code != 200:
    raise Exception(f"Failed to fetch CIDR list from {url}")

# Process the CIDR list, ignoring commented lines
cidr_list = [line.strip() for line in response.text.splitlines() if line.strip() and not line.startswith("#")]

# Expand CIDR ranges into individual IPs
def expand_cidr(cidr):
    return [str(ip) for ip in IPNetwork(cidr)]

expanded_ips = []
for cidr in cidr_list:
    expanded_ips.extend(expand_cidr(cidr))

# Create a DataFrame
df = pd.DataFrame(expanded_ips, columns=['ip_address'])

# Perform DNS and Hostname Lookup
def resolve_dns(ip):
    try:
        answers = dns_query.dns_lookup(ip).answer
        return answers[0] if answers else None
    except Exception:
        return None

def resolve_host(ip):
    try:
        return socket.gethostbyaddr(ip)[0]
    except Exception:
        return None

df['DNS'] = df['ip_address'].apply(resolve_dns)
df['Host'] = df.apply(lambda row: resolve_host(row['ip_address']) if not row['DNS'] else row['DNS'], axis=1)

# Geolocation functions
def get_latitude(ip):
    try:
        response = geoip_reader.city(ip)
        return response.location.latitude
    except Exception:
        return None

def get_longitude(ip):
    try:
        response = geoip_reader.city(ip)
        return response.location.longitude
    except Exception:
        return None

def get_country(ip):
    try:
        response = geoip_reader.city(ip)
        return response.country.iso_code
    except Exception:
        return None

# Add geolocation data
df['Latitude'] = df['ip_address'].apply(get_latitude)
df['Longitude'] = df['ip_address'].apply(get_longitude)
df['Country'] = df['ip_address'].apply(get_country)

# Drop rows with missing geolocation data
df = df.dropna(subset=['Latitude', 'Longitude'])

# Save to CSV
df.to_csv('cidr_to_dns_host.csv', index=False)

# Print the top 10 countries
print(df['Country'].value_counts().head(10))

# Create a map with geolocated IPs
map = folium.Map(location=[20.0, 90.0], tiles='cartodb positron', zoom_start=6)
mcluster = MarkerCluster().add_to(map)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"IP: {row['ip_address']}\nDNS: {row['DNS']}\nHost: {row['Host']}"
    ).add_to(mcluster)

map.save("index.html")
map

Fetching CIDR list from https://raw.githubusercontent.com/threatcode/proxy-list/refs/heads/master/COUNTRY_CIDR/BD_CIDR.txt...
Processing CIDR ranges...
GeoIP lookup completed for 1000 IPs
Top 10 Countries:
BD: 500
IN: 300
US: 100


<a href="https://colab.research.google.com/github/threatcode/cidr_map/blob/main/cidr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>